## Librerías

In [1]:
##
import numpy as np
"""tratar imagenes"""
import cv2
"""crear graficas"""
import matplotlib.pyplot as plt
"""manipular rutas del sistema operativo"""
import os
"""encontrar directorios"""
import pathlib
"""mover archivos"""
import shutil
"""operaciones avanzadas con arrays de imagenes"""
import scipy
"""manipular imagenes"""
import PIL
"""invocar metodos avanzados"""
import tensorflow as tf
"""importar modulos de keras"""
from tensorflow import keras
"""generador de datos"""
from tensorflow.keras.preprocessing.image import ImageDataGenerator
"""conversor y carga de datos"""
from keras.utils import  array_to_img, img_to_array, load_img
"""eliminar fondo de imagenes"""
from rembg import remove

# Cargar conjunto de datos

In [2]:
"""directorio de imagenes"""
dir_img = "../../Segmentacion/"
"""comprobar directorio principal de imagenes"""
dataimg_dir = pathlib.Path(dir_img)
"""obtener clases de las imagenes"""
clases_img = os.listdir(dataimg_dir)
"""mostrar clases"""
print(clases_img)
"""mostrar directorio local"""
dataimg_dir

['ED3', 'ED6', 'EDH']


WindowsPath('../../Segmentacion')

In [3]:
"""crear diccionario para el acceso de imagenes"""
img_subset = {
    'ed3': list(dataimg_dir.glob('ED3/*')),
    'ed6': list(dataimg_dir.glob('ED6/*')),
    'edh': list(dataimg_dir.glob('EDH/*'))
}

"""crear diccionario de etiquetas"""
cacao_etiquetas = {
    'ed3': 0,
    'ed6':1,
    'edh':2
}

In [4]:
"""directorio de imagenes"""
dsave_imager1 = "../../Redimension1/"
"""comprobar directorio principal de imagenes"""
maindir_r1 = pathlib.Path(dsave_imager1)
"""diccionario de almacenamiento"""
dict_saver1 = {
    'ed3': list(maindir_r1.glob('ED3/')),
    'ed6': list(maindir_r1.glob('ED6/')),
    'edh': list(maindir_r1.glob('EDH/'))
}

In [5]:
def comprobarImagenes(_clases_img,_dataimg_dir):
    """
    Comprobar cantidad de imagenes por directorio o clase. 
    """
    for i in _clases_img:
        """mostrar n imagen"""
        print(list(_dataimg_dir.glob(f'./{i}/*.jpg'))[:5])
        """mostrar cantidad imagen"""
        print(len(list(_dataimg_dir.glob(f'./{i}/*.jpg'))))

    print("Total imagenes:",len(list(_dataimg_dir.glob('*/*.jpg'))))

In [ ]:
#mostrar el total de imágenes a procesar
comprobarImagenes(clases_img,dataimg_dir)

## Dimensiones de images

In [176]:
"""funcion para observar la dimension de las imagenes"""
def verificarDimensionImg(_cacaoimgdict):
    """acceder al conjunto de imagenes"""
    for clase,imgs_cacao in _cacaoimgdict.items():
        """mostrar clase de cacao"""
        print('Clase:',clase)
        """acceder a cada img"""
        for imagen in range(len(imgs_cacao)):
            """mostrar clase de cacao, nombre imagen, dimension (y, x, RGB)"""
            print(f"{str(_cacaoimgdict[clase][imagen])}:{cv2.imread(str(_cacaoimgdict[clase][imagen])).shape}")

In [ ]:
verificarDimensionImg(img_subset)

In [178]:

def cambiarDimensionImg(_dictimgs:dict, _dictsave:dict, ancho=0, alto=0):
    """Cambiar dimension de imagenes"""
    """seleccionar clase e imagenes a partir del diccionario de imgs""" 
    for (clase, images), (ksave) in zip(_dictimgs.items(), _dictsave.values()):
        """mostrar clase cacao"""
        print('Clase:',clase)
        """recorrer el conjunto de imagenes"""
        for imagen in range(len(images)):
            """obtener dimension actual de img con todos los canales x y rgb"""
            dimen_img = cv2.imread(str(_dictimgs[clase][imagen]),cv2.IMREAD_UNCHANGED)
            """cambiar dimension de img"""
            imagen_redim = cv2.resize(dimen_img,(ancho,alto))
            """sobreescribir img"""
            print(f'{ksave[0]}/'+str(clase).upper()+ f'_{imagen+1}.jpg')
            cv2.imwrite(f'{ksave[0]}/'+str(clase).upper()+ f'_{imagen+1}.jpg', imagen_redim)

In [ ]:
"""NOTA: DEBO MEJORAR ESTA FUNCION PARA GUARDAR PROCESOS"""
cambiarDimensionImg(img_subset, dict_saver1, 960, 540)

## Aumentar Imágenes (DA)

In [6]:
"""Objeto para generar imágenes por
rango de rotacion: como quiero girar la imagen
rango de cambio de ancho y por altura: mover el foco
reescalado:
rango de corte: corte de la imagen
Aproximacion y volteado de imagen: generar la imagen
"""
generador_imagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0,
        height_shift_range=0,
        rescale=None,
        shear_range=0.0,
        zoom_range=0.0,
        horizontal_flip=True,
        #vertical_flip=True,
        #channel_shift_range=5,
        fill_mode='nearest'
        )

In [22]:
"""clase para aumentar imagenes"""
class AumentarImagen(object):

    """metodo para crear imagen"""
    def crearImagen(self, _generador_imagen, _ruta, _cantidad=0):
        """asignar directorio"""
        os.chdir(_ruta)
        j=1
        """acceder a imagenes de directorio"""
        for dirimg in os.listdir():
            """seleccionar imagen"""
            img = load_img(f"{dirimg}")
            """obtener imagen en array original"""
            x = img_to_array(img)
            """reformar imagen"""
            x = x.reshape((1,) + x.shape)
            """contador para crear imagen"""
            i = 0
            """generar nueva imagen con objeto encapsulado"""
            for bt in _generador_imagen.flow(x, 
                                            batch_size=1,
                                            save_to_dir=_ruta,
                                            save_prefix= dirimg[:-4]+f'_da', 
                                            save_format='jpg'):
                i += 1
                """crear n imagenes de 1 img.jpg"""
                if i == _cantidad:     
                    break
            print(f"{j} : {dirimg}")
            j+=1

def eliminardFondo(ruta_img_fondo):
    """
    Metodo para eliminar el fondo de las imagenes.
    
    imagenes_actuales: lista de imágenes en el directorio.
    j: contador de imagen en el directorio.
    imagen: contenido.
    ruta_completa_imagen: ruta de la image.
    d_img: lectura de la imagen
    _, buffer: conversion de la imagen a png.
    remover_fondo: eliminar fondo de la imagen.
    output_image_path: guardar la imagen procesada.
    """
    imagenes_actuales = os.listdir(ruta_img_fondo)
    print(imagenes_actuales)
    
    #bloque de procesamiento de imagenes
    j = 1
    for imagen in imagenes_actuales:
        ruta_completa_imagen = os.path.join(ruta_img_fondo, imagen)
        d_img = cv2.imread(ruta_completa_imagen)

        if d_img is None:
            print(f"Error al cargar la imagen: {imagen}")
            continue

        _, buffer = cv2.imencode('.png', d_img)
        remover_fondo = remove(buffer.tobytes())

        #apertura y escritura de la imagen sin fondo
        output_image_path = os.path.join(ruta_img_fondo, f"{os.path.splitext(imagen)[0]}_sf{j}.png")
        with open(output_image_path, 'wb') as output_file:
            output_file.write(remover_fondo)

        print(f"{j}: {imagen} procesada.")
        j += 1

In [24]:
ruta="../../Redimension1/EDH/"

"""crear objeto para aumentar imagenes"""
aumentar_imagen = AumentarImagen()
"""crear imagenes"""
aumentar_imagen.crearImagen(generador_imagen,ruta, 19)

1 : EDH_1.jpg
2 : EDH_10.jpg
3 : EDH_10_da_0_1121.jpg


KeyboardInterrupt: 

## Eliminar el fondo

In [23]:
#direccion de la ruta
ruta = "../../Redimension1/EDH/"
#invocar metodo para eliminar el fondo de cada imagen
aumentar_imagen.eliminardFondo(ruta)

AttributeError: 'AumentarImagen' object has no attribute 'eliminardFondo'